In [1]:
import numpy as np
import pandas as pd


In [19]:
# load the trip data set
data_dir = './data/'
df_trip = pd.read_csv('V:\\Asset Management Program\\Data Science\\Data\\Trip_Household_Merged.csv', low_memory=False)

# Filter trips that end outside seattle
df_trip = df_trip[df_trip['uv_origin'] != "Outside Seattle"]

# clean errors
#df_trip = df_trip[df_trip['trip_path_distance'] != " "]
df_trip['trip_path_distance'] = df_trip['trip_path_distance'].astype(float)

# create bins, apply weights
df_trip['dist_under_2'] = np.where(df_trip['trip_path_distance']<2, df_trip['trip_wt_final'], 0)
df_trip['dist_2_to_10'] = np.where((df_trip['trip_path_distance']>=2) & 
                                   (df_trip['trip_path_distance']<10), df_trip['trip_wt_final'], 0)
# calculate census tract
df_trip['tract'] = df_trip['bg_origin'].astype(str).apply(lambda x: x[:9])
                          
# aggregate by blockgroup origin
df_proximity = df_trip.groupby(['tract'], as_index=False).agg({'dist_under_2':sum,
                                                                    'dist_2_to_10':sum,
                                                                  'bg_dest':'count'})
# filter blockgroups with trips less than 25
df_proximity = df_proximity[df_proximity['bg_dest'] >= 100]
df_proximity['proximity_ratio'] = df_proximity['dist_under_2']/df_proximity['dist_2_to_10']

# remove outliers
df_proximity = df_proximity[df_proximity['proximity_ratio'] < 6]


df_proximity = df_proximity.reset_index(drop=True)
print (df_proximity)
df_proximity.to_csv('V:\\Asset Management Program\\Data Science\\Data\\Proximity_Ratio.csv', mode='w', header=True, index=False)


        tract  dist_under_2  dist_2_to_10  bg_dest  proximity_ratio
0   530330001  10328.365437  11900.151717      146         0.867919
1   530330004   7693.770448   7545.888617      343         1.019598
2   530330006  12044.093067  39586.953019      174         0.304244
3   530330007   2600.046696   7574.386992      395         0.343268
4   530330012  12956.382692  27226.628342      451         0.475872
5   530330013   3427.068778   3610.793207      157         0.949118
6   530330016   5979.753801   5622.356621      107         1.063567
7   530330017  13707.611589  14087.109868      268         0.973061
8   530330018   3443.767945   9471.808855      315         0.363581
9   530330019   8351.600472  16086.525641      191         0.519167
10  530330026  10669.383109   7676.895672      320         1.389804
11  530330030   7692.010067   3812.649916      140         2.017497
12  530330032  14249.670940   9318.448361      245         1.529189
13  530330033  21543.037880   6349.468521      3

In [68]:
# evaluate results
from sklearn.metrics import mean_squared_error

target = df_proximity['proximity_ratio']
predictions = df_proximity['proximity_ratio']

mse = mean_squared_error(target, predictions)
print ("mean_squared_error: ", mse)

mean_squared_error:  0.0


In [9]:
origin = 234346672345
print (str(origin)[0:9])

234346672
